In [10]:
import numpy as np
import os
from glob import glob

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, MaxPooling2D, UpSampling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import Input, concatenate, Activation, Dense, Reshape, Lambda, Dropout, Multiply, Add
from tensorflow.keras import models

import imgaug
from sklearn.model_selection import train_test_split

from utils_dh import utils, unet
from python.common.NnetsX import NNets ,MyReLU

import cv2
import math

import datetime
import pickle as pk

import matplotlib.pyplot as plt
import time

In [2]:
gpu = 0
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.set_visible_devices(gpus[gpu], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[gpu], True)
    except RuntimeError as e:
        print(e)

tf.keras.backend.set_image_data_format('channels_last')

BATCH_NORMALIZATION_NO = -1
BATCH_NORMALIZATION_YES = 0

DOWNSAMPLING_MAXPOOL = 0
DOWNSAMPLING_STRIDED_CONV = 1

UPSAMPLING_UPSAMPLE = 0
UPSAMPLING_TRANSPOSED_CONV = 1

batch_size = 8
epochs = 300
n_channel = 4
input_shape = (256,256,n_channel)

model_dir = '/docker/FluSegTip_entire/models/1101_064217/model_best.h5'


nnets = NNets()
nnets.m_SamePartActivation = MyReLU
nnets.m_RegularizerL1L2 = False
nnets.m_Dropout = 0.5
nnets.m_Residual = True
nnets.m_BatchNormalization = BATCH_NORMALIZATION_YES
nnets.m_BorderMode = "same"
nnets.m_Initialization = "glorot_uniform"
nnets.m_DownSampling = DOWNSAMPLING_STRIDED_CONV
nnets.m_UpSampling = UPSAMPLING_UPSAMPLE
nbUsedChannel = n_channel
nbStartFilter = 8
kernelSize = 3
nbConvPerLayer = [2, 2, 2, 2, 2, 2, 2]
nbDeconvPerLayer = [2, 2, 2, 2, 2, 2]

optimizer = SGD(lr=0.01, decay=5e-4, momentum=0.99)

model1 = nnets.DefineDeepUVNet(input_shape, _nbFilters=nbStartFilter, _kernelSize=kernelSize,
                              _convPerLevel=nbConvPerLayer, _upConvPerLevel=nbDeconvPerLayer, _optimizer=optimizer)

2021-11-04 02:05:41.663735: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-04 02:05:41.670845: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-04 02:05:41.671960: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
/usr/local/lib/python3.8/dist-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
2021-11-04 02:05:41.683207: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to 

In [3]:
model2 = unet.unet((256,256,16), layers=6, filter_root=64, output_channel=2, residual=1, conv_act='leaky')

In [5]:
input_tensor = Input((256,256,19))

gw4 = Lambda(lambda x: x[:,:,:,:4], output_shape=(1,))(input_tensor)
angios = Lambda(lambda x: x[:,:,:,4:], output_shape=(1,))(input_tensor)

seg_gw_not = model1(gw4)
bitcast_to_int32 = tf.bitcast(seg_gw_not, tf.int32)
invert_bits = tf.constant(-1) - bitcast_to_int32
bitcast_to_float = tf.bitcast(invert_bits, tf.float32)
seg_gw = tf.math.scalar_mul(2.0, bitcast_to_float)
seg_gw = tf.subtract(seg_gw, 1)

to_reg = concatenate([seg_gw, angios], axis=-1)
output = model2(to_reg)
new_model = models.Model(input_tensor, output)

In [7]:
new_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 256, 256, 19 0                                            
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 256, 256, 4)  0           input_4[0][0]                    
__________________________________________________________________________________________________
model (Functional)              (None, 256, 256, 1)  14181201    lambda_2[0][0]                   
__________________________________________________________________________________________________
tf.bitcast_2 (TFOpLambda)       (None, 256, 256, 1)  0           model[1][0]                      
____________________________________________________________________________________________

In [11]:
del model1, model2

In [8]:
tin = np.random.rand(1000, 256, 256, 19).astype(np.float32)

In [13]:
times = []
pre = time.time()
ppre = pre
for i in range(tin.shape[0]):
    data = tin[i].reshape(1, 256, 256, 19)
    _ = new_model.predict(data)
    t = time.time() - pre
    times.append(t)
    pre = time.time()
last = time.time()
times = np.array(times)
print(np.mean(times), np.std(times), (last - ppre)/1000.0)

0.055670574426651 0.0091847843981798 0.05567415904998779


In [18]:
1 / (np.mean(times)+0.032)

11.40633566666822

In [19]:
1 / (((last - ppre)/1000.0) + 0.032)

11.405869310133284